In [ ]:
# Developed by James Simkins 
# with help from holoviz gitter community

In [ ]:
import xarray as xr #version 0.15.1
import hvplot.xarray #version 0.6.0
import holoviews as hv
import geoviews as gv #version 1.8.1
import cartopy.crs as ccrs
import numpy as np
gv.extension('bokeh')

clickedValues = []

# convert to numpy
def great_circle(lon1, lat1, lon2, lat2):
    # REF: https://medium.com/@petehouston/calculate-distance-of-two-locations-on-earth-using-python-1501b1944d97
    # Convert to numpy
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])


    dist = 6371 * (
        np.arccos(np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon1 - lon2))
    )
    
    return dist

def plot(x, y):
    lon = x
    lat = y

    if x is not None:
        
        # Curvilinear
        d = great_circle(lon, lat, da['lon'], da['lat'])
        ind = np.nonzero(np.equal(d, np.amin(d)))
        yloc = int(ind[0][0])
        xloc = int(ind[1][0])
        
        clickedValues.append([lon, lat, yloc, xloc])

        da_sel = da.sel(y = yloc, x = xloc)        
        da.loc[yloc, xloc] = 0

    # This has a side effect of dynamically changing the colorbar and the
    # rendered image.
    
    # REF: https://hvplot.holoviz.org/user_guide/Geographic_Data.html#declaring-an-output-projection
    plt = da.hvplot.quadmesh(
        'lon', 'lat', projection=ccrs.Orthographic(-90, 90),
        global_extent=True, frame_height=540, cmap='viridis',
        coastline=True
    )
    
    return plt

ds = xr.tutorial.open_dataset("rasm").isel(time=0)
da = ds["Tair"]

da = da.rename({
    'xc': 'lon',
    'yc': 'lat'
})

tap_stream = hv.streams.Tap()
dmap = gv.DynamicMap(plot, streams=[tap_stream])
dmap